# AppScraper Workflow: From fetching the apps to scoring by subcategories

#### this notebook uses the library AppStoreScraper taken from the repository https://github.com/digitalmethodsinitiative/itunes-app-scraper 

### Libraries

In [1]:
import pandas as pd

from urllib.parse import quote_plus
from itunes_app_scraper.util import AppStoreException, AppStoreCollections, AppStoreCategories, AppStoreMarkets

# for scraping app info from App Store
from itunes_app_scraper.scraper import AppStoreScraper

# for scraping app reviews from App Store
from app_store_scraper import AppStore

# for pretty printing data structures
from pprint import pprint

# for keeping track of timing
from datetime import datetime
#from tzlocal import get_localzone

# for building in wait times
import random
import time
import json
import os.path, time
import numpy as np
import requests
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


from gspread_pandas import Spread
from gspread_pandas.conf import get_config as gspread_get_config
__google_key_path = Path.cwd() / 'swsp-missions.json'
print(gspread_get_config(file_name=__google_key_path)['client_email'])

mission-management@cellular-block-264708.iam.gserviceaccount.com


### Connection with the spreadsheet

In [3]:
research_ui = Spread('1XrtpyON9eb13u6dZcgLL_WID3AVt-j8_ROaNHbCj2Dk')
# Set a new case_specific_ui everytime you work with a new subcategory!!!
case_specific_ui = Spread('1yEkcyHkSscmi6rwtFtMsheysU7H4eXmNx3jmANsPAIw') #Job
creative_ui = Spread('1pDI-aB-0tH4FA_OxycdgMONxqglVLGNXmhS_2JEwYYI')

ui_urls = {'research_ui' : research_ui.url,
           'creative_ui' : creative_ui.url,
           'case_specific_ui' : case_specific_ui.url}
ui_urls

{'research_ui': 'https://docs.google.com/spreadsheets/d/1XrtpyON9eb13u6dZcgLL_WID3AVt-j8_ROaNHbCj2Dk',
 'creative_ui': 'https://docs.google.com/spreadsheets/d/1pDI-aB-0tH4FA_OxycdgMONxqglVLGNXmhS_2JEwYYI',
 'case_specific_ui': 'https://docs.google.com/spreadsheets/d/1yEkcyHkSscmi6rwtFtMsheysU7H4eXmNx3jmANsPAIw'}

### Folders: set the theme of the apps you want to scrape and name the collection folder accordingly to the project

In [4]:
theme = 'Job_apps'

In [5]:
path_folder = Path.cwd()
path_cache_folder = path_folder / f'{theme}_AppScraper'
path_applyzer_countries = path_folder / 'applyzer_countries.csv'
path_applyzer_markets_ios = path_folder / 'applyzer_markets_ios.csv'
path_results = path_folder / 'results'

if not path_cache_folder.exists():
    Path.mkdir(path_cache_folder)
    
if not path_results.exists():
    Path.mkdir(path_results)

### Functions

In [6]:
def get_app_details(app_id, country, lang, flatten=True):
    try:
        app_id = int(app_id)
        id_field = "id"
    except ValueError:
        id_field = "bundleId"

    url = "https://itunes.apple.com/lookup?%s=%s&country=%s&entity=software" % (id_field, app_id, country)
    print(f'Getting data for {app_id} in {country}')

    try:
        result = requests.get(url).json()
    except Exception:
        try:
            # handle the retry here. 
            # Take an extra sleep as back off and then retry the URL once. 
            time.sleep(5)
            result = requests.get(url).json()
        except Exception:
            raise AppStoreException("Could not parse app store response for ID %s" % app_id)

    try:
        app = result["results"][0]
    except IndexError:
        app = dict(result)
            
        #raise AppStoreException("No app found with ID %s" % app_id)
    

    # 'flatten' app response
    # responses are at most two-dimensional (array within array), so simply
    # join any such values
    if flatten:
        for field in app:
            if isinstance(app[field], list):
                app[field] = ",".join(app[field])

    return app

def _log_error(app_store_country, message):
    """
    Write the error to a local file to capture the error. 
    :param str app_store_country: the country for the app store
    :param str message: the error message to log
    """
    app_log = "{0}_log.txt".format(app_store_country)
    errortime = datetime.now().strftime('%Y%m%d_%H:%M:%S - ')
    fh = open(app_log, "a")
    fh.write("%s %s \n" % (errortime,message))
    fh.close()
    
    
def get_multiple_app_details(app_ids, country, lang):
    """
    Get app details for a list of app IDs
    :param list app_id:  App IDs to retrieve details for
    :param str country:  Two-letter country code for the store to search in.
                         Defaults to 'nl'.
    :param str lang: Dummy argument for compatibility. Unused.
    :return generator:  A list (via a generator) of app details
    """
    for app_id in app_ids:
        try:
            time.sleep(1)
            yield get_app_details(app_id, country=country, lang=lang)
        except AppStoreException as ase:
            _log_error(country, str(ase))
            continue

In [7]:
def download_apps_country(app_name, country, search):
    scraper = AppStoreScraper()
    results = scraper.get_app_ids_for_query(app_name, num=search, country=country, lang='en')
    app_details = list(get_multiple_app_details(results, country, lang='en'))
    df_results = pd.DataFrame(app_details)
    path_cache_response = path_cache_folder / f'{country}_{app_name}_{search}.csv'
    with open(path_cache_response, 'w') as outfile:
        df_results.to_csv(path_cache_response, index=False)


def fetch_response(app_name, country_list, search):
    for country in country_list:
        path_cache_response = path_cache_folder / f'{country}_{app_name}_{search}.csv'
        if not path_cache_response.exists():
            download_apps_country(app_name, country, search)
            time.sleep(2)
            print("downloaded file for", path_cache_response.name)
        else:
            print("loaded file for", path_cache_response.name)
            

In [8]:
def construct_df(file):
    df_raw = pd.read_csv(file)
    df_raw['search_details'] = file.name.replace('.csv',"")
    df_raw['country'] = df_raw['search_details'].str[:2].str.upper()
    df_raw['search_details'] = df_raw['search_details'].str[3:]
    df_raw['downloaded_on'] = time.ctime(os.path.getctime(file))
    return df_raw

def final_df(folder):
    df = None

    for p in Path('.', folder).iterdir():
        print(p.name)
        if p.is_file():
            if df is None:
                df = construct_df(p)
            else:
                df = df.append(construct_df(p))
    return df

In [9]:
def df_results():
    df = None
    for p in Path('.', path_results).iterdir():
        if p.is_file():
            if df is None:
                df = pd.read_csv(p)
            else:
                df = df.append(pd.read_csv(p))
    return df.reset_index(drop=True)

### Import Country selection from research_ui

In [10]:
df_countries = research_ui.sheet_to_df(index=False, sheet='OUTPUT - Countries_Selection')
df_country_ids = pd.read_csv(path_applyzer_countries)
df_countries = (
    df_countries.set_index('ISO2')
    .join(df_country_ids[['id', 'code']].set_index('code'))
    .reset_index()
)
print(f'There are {len(df_countries)} countries')
df_countries.head()

There are 57 countries


,ISO2,Internal Country,ISO3,population_2020,internet_penetration,internet_users,id
0,AR,Argentina,ARG,45376763,74.29%,33712623.81,2
1,AU,Australia,AUS,25687041,86.55%,22230862.18,3
2,AT,Austria,AUT,8917205,87.75%,7825043.993,4
3,BE,Belgium,BEL,11555997,90.37%,10443213.78,5
4,BR,Brazil,BRA,212559409,70.43%,149714694.7,6


## Step 1:
### Fetching data by keywords with AppStoreScraper()

### Set the keyword under the variable "app_name"

In [11]:
country_list = df_countries['ISO2'].str.lower()
search = 20
app_name = 'indeed'

In [12]:
fetch_response(app_name, country_list, search)

loaded file for ar_indeed_20.csv
loaded file for au_indeed_20.csv
loaded file for at_indeed_20.csv
loaded file for be_indeed_20.csv
loaded file for br_indeed_20.csv
loaded file for bg_indeed_20.csv
loaded file for ca_indeed_20.csv
loaded file for cl_indeed_20.csv
loaded file for cn_indeed_20.csv
loaded file for co_indeed_20.csv
loaded file for cz_indeed_20.csv
loaded file for dk_indeed_20.csv
loaded file for eg_indeed_20.csv
loaded file for ee_indeed_20.csv
loaded file for fi_indeed_20.csv
loaded file for fr_indeed_20.csv
loaded file for de_indeed_20.csv
loaded file for gr_indeed_20.csv
loaded file for hk_indeed_20.csv
loaded file for hu_indeed_20.csv
loaded file for in_indeed_20.csv
loaded file for id_indeed_20.csv
loaded file for ie_indeed_20.csv
loaded file for il_indeed_20.csv
loaded file for it_indeed_20.csv
loaded file for jp_indeed_20.csv
loaded file for ke_indeed_20.csv
loaded file for lt_indeed_20.csv
loaded file for my_indeed_20.csv
loaded file for mx_indeed_20.csv
loaded fil

In [13]:
df_apps = final_df(path_cache_folder)
df_apps.to_csv(f'df_{theme}_apss_tot.csv', index=True)

ae_job search_100.csv
gb_job search_100.csv
ua_job search_100.csv
us_job search_100.csv
vn_job search_100.csv
ar_job search_100.csv
au_job search_100.csv
at_job search_100.csv
be_job search_100.csv
br_job search_100.csv
ca_job search_100.csv
cl_job search_100.csv
cn_job search_100.csv
co_job search_100.csv
cr_job search_100.csv
hr_job search_100.csv
cz_job search_100.csv
dk_job search_100.csv
de_job search_100.csv
sv_job search_100.csv
es_job search_100.csv
fi_job search_100.csv
fr_job search_100.csv
gr_job search_100.csv
gt_job search_100.csv
hk_job search_100.csv
hu_job search_100.csv
in_job search_100.csv
id_job search_100.csv
ie_job search_100.csv
il_job search_100.csv
it_job search_100.csv
kr_job search_100.csv
kw_job search_100.csv
lb_job search_100.csv
lu_job search_100.csv
my_job search_100.csv
mx_job search_100.csv
nl_job search_100.csv
nz_job search_100.csv
no_job search_100.csv
pk_job search_100.csv
pa_job search_100.csv
pe_job search_100.csv
ph_job search_100.csv
pl_job sea

#### Keywords for the theme

In [14]:
list(set(df_apps['search_details'].tolist()))

['find a job_20', 'glassdoor_20', 'indeed_20', 'job search_100']

## Step 2
### Case specific:
### Importing the desired df_apps to normalise it and do some basic filtering 

In [15]:
theme = theme
df_apps = pd.read_csv(f'df_{theme}_apss_tot.csv')

In [16]:
df_apps_tot = df_apps.copy()
df_apps_tot = df_apps_tot.reset_index()
df_apps_tot = df_apps_tot.dropna(subset = ["trackId"])
df_apps_tot = df_apps_tot.loc[df_apps_tot['price'] == 0.00]
df_apps_tot['ranking'] = df_apps_tot['Unnamed: 0'] + 1
df_apps_tot = df_apps_tot.drop(['Unnamed: 0'], axis=1)
df_apps_tot = df_apps_tot[['trackId', 'trackName', 'sellerName', 'country','ranking','trackViewUrl', 'search_details', 'primaryGenreName',
             'genres', 'genreIds', 'primaryGenreId', 'userRatingCount','averageUserRating']]
df_apps_tot['trackId'] = df_apps_tot['trackId'].astype(int)
df_apps_tot['primaryGenreId'] = df_apps_tot['primaryGenreId'].astype(int)
df_apps_tot['chart'] = 'free'

In [17]:
df_categories = (
    df_apps_tot[['primaryGenreId', 'primaryGenreName']]
    .groupby(['primaryGenreId', 'primaryGenreName'])
    .agg({'primaryGenreName':'count'})
    .rename(columns={'primaryGenreName':'counts'})
    .sort_values(by='counts', ascending=False)
    .reset_index()
)

df_categories_applyzer = pd.read_csv(path_applyzer_markets_ios)


df_categories = (
    df_categories.set_index('primaryGenreName')
    .join(df_apps_tot[['primaryGenreName', 'search_details', 'primaryGenreId']]
    .groupby(['search_details', 'primaryGenreName']).count()
    .unstack().transpose().droplevel(0)).reset_index()
    .rename(columns={'counts': 'tot_apps'})
)


df_categories = df_categories.fillna(0).set_index('primaryGenreName').astype(int)
case_specific_ui.df_to_sheet(df_categories, sheet=f'SOURCE - Categories Overview', index=True, replace=True)
df_categories

,primaryGenreId,tot_apps,find a job_20,glassdoor_20,indeed_20,job search_100
primaryGenreName,,,,,,
Business,6000,7456,644,882,952,4978
Games,6014,930,344,0,7,579
Productivity,6007,474,15,0,103,356
Lifestyle,6012,278,44,39,19,176
Education,6017,179,24,0,10,145
Utilities,6002,109,22,3,25,59
Social Networking,6005,87,0,25,1,61
Reference,6006,69,0,0,3,66
Photo & Video,6008,59,25,0,0,34


In [18]:
df_apps_normalized = (
    df_apps_tot.set_index('country').join(df_countries[['ISO2', 'id', 'Internal Country', 'internet_users',
       'internet_penetration']].set_index('ISO2'))
    .reset_index()
    .rename(columns={
        'index' : 'ISO2',
        'trackId' : 'app_id',
        'trackName' : 'name',
        'sellerName' : 'seller_name',
        'trackViewUrl' : 'url',
        'primaryGenreName' : 'cat_name',
        'id' : 'country_id',
        'primaryGenreId' : 'category_id'
        
    })
    [['app_id', 'ranking', 'chart', 'country_id', 'category_id', 'name',
       'seller_name', 'url', 'Internal Country',  'internet_users',
       'internet_penetration','cat_name', 'search_details']]
)


df_apps_normalized = df_apps_normalized.loc[df_apps_normalized['Internal Country'].isnull() != True][['app_id', 'ranking', 'chart', 'country_id', 'category_id', 'name',
       'seller_name', 'url', 'Internal Country',  'internet_users',
       'internet_penetration','cat_name', 'search_details']]

df_apps_normalized

,app_id,ranking,chart,country_id,category_id,name,seller_name,url,Internal Country,internet_users,internet_penetration,cat_name,search_details
0,724241430,1,free,58.0,6000,Naukrigulf Job Search App,Info Edge (India) Ltd,https://apps.apple.com/ae/app/naukrigulf-job-s...,UAE,9806331.398,99.15%,Business,job search_100
1,309735670,2,free,58.0,6000,Indeed Job Search,Indeed Inc.,https://apps.apple.com/ae/app/indeed-job-searc...,UAE,9806331.398,99.15%,Business,job search_100
2,451816577,3,free,58.0,6002,UAE Jobs,Muhammad Ameen Ameerzada,https://apps.apple.com/ae/app/uae-jobs/id45181...,UAE,9806331.398,99.15%,Utilities,job search_100
3,1532042887,4,free,58.0,6000,HiresGulf,HORIZON INTERNATIONAL RECRUITMENT SERVICES - S...,https://apps.apple.com/ae/app/hiresgulf/id1532...,UAE,9806331.398,99.15%,Business,job search_100
4,1093457564,5,free,58.0,6000,GulfTalent - Job Search App,Gulf Talent FZ-LLC,https://apps.apple.com/ae/app/gulftalent-job-s...,UAE,9806331.398,99.15%,Business,job search_100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9746,1069568033,16,free,52.0,6000,hokify Job App - Find Jobs,JobSwipr GmbH,https://apps.apple.com/za/app/hokify-job-app-f...,South Africa,33312145.87,56.17%,Business,indeed_20
9747,1040443949,17,free,52.0,6000,Reed.co.uk Job Search,Reed Online Ltd,https://apps.apple.com/za/app/reed-co-uk-job-s...,South Africa,33312145.87,56.17%,Business,indeed_20
9748,1338484219,18,free,52.0,6000,Jobs on map - easy job search,Andrei Sadovnicov,https://apps.apple.com/za/app/jobs-on-map-easy...,South Africa,33312145.87,56.17%,Business,indeed_20
9749,1155800838,19,free,52.0,6000,Make My Resume,Sagar Joshi,https://apps.apple.com/za/app/make-my-resume/i...,South Africa,33312145.87,56.17%,Business,indeed_20


In [19]:
df_appids_details = (
    df_apps_normalized.groupby(['cat_name','app_id',  'name'])
    .agg({'search_details': lambda x: ','.join(sorted(pd.Series.unique(x))), 
          'category_id': 'count', 
          'Internal Country':pd.Series.nunique,
         'url' : list})
    .sort_values(by=['cat_name', 'category_id'], ascending=[True, False])
    .rename(columns={'category_id': 'counts',
                    'Internal Country' : 'country_counts',
                    'url':'urls'})
    .reset_index()
)

print(f'There are {df_appids_details.app_id.nunique()} unique apps, over {len(df_appids_details)} entries')
case_specific_ui.df_to_sheet(df_appids_details, sheet=f'SOURCE - App_Details', index=False, replace=True)
#df_appids_details#.loc[(df_appids_details.cat_name == 'Graphics & Design')]

There are 626 unique apps, over 913 entries


### Filtering (Case specific): Include column

In [20]:
df_exclude_select = case_specific_ui.sheet_to_df(sheet="OUTPUT - Exclude list", index=False)
exclude_cat_list = df_exclude_select['exclude_cat'].tolist()
exclude_names_list = df_exclude_select['exclude_names'].tolist()
include_names_list = df_exclude_select['include_names'].tolist()

In [21]:
exclude_cat_list

['Games',
 'Photo & Video',
 'Travel',
 'Entertainment',
 'Navigation',
 'Food & Drink',
 'Music',
 '',
 '',
 '',
 '',
 '']

In [22]:
exclude_names_list

['Curriculum',
 'CV',
 'cvs',
 'Cvs',
 'Resume',
 'resume',
 'Builder',
 'Lebenslauf',
 'builder',
 'templates',
 '',
 '']

In [23]:
include_names_list

['job',
 'search',
 'find',
 'hire',
 'jobs',
 'JOBS',
 'Job',
 'Jobs',
 'Search',
 'SEARCH',
 'Find',
 'Hire']

In [24]:
df_appids_details.loc[df_appids_details.cat_name.isin(exclude_cat_list), 'Include'] = 'False'
print(f"False = {len(df_appids_details.loc[(df_appids_details['Include'] == 'False')])}")

False = 124


In [25]:
df_false = df_appids_details.loc[df_appids_details['Include'] == 'False']
df_examine = df_appids_details.loc[df_appids_details['Include'] != 'False']

In [26]:
df_examine.loc[(df_examine['name'].str.contains('job', case=False))
               | (df_examine['name'].str.contains('search', case=False))
               | (df_examine['name'].str.contains('find', case=False))
               | (df_examine['name'].str.contains('hire', case=False))
               | (df_examine['name'].str.contains('jobs', case=False)), 'Include'] = 'True'

df_examine.loc[(df_examine['Include'] != 'False') 
                      & (df_examine['name'].str.contains('CV', case=False))
                      | (df_examine['name'].str.contains('Cvs', case=False))
                      | (df_examine['name'].str.contains('Resume', case=False))
                      | (df_examine['name'].str.contains('builder', case=False))
                      | (df_examine['name'].str.contains('Lebenslauf', case=False)), 'Include'] = 'False'



print(f"True = {len(df_examine.loc[(df_examine['Include'] == 'True')])}, False = {len(df_examine.loc[(df_examine['Include'] == 'False')])}")
df_include = df_examine.append(df_false)
print(f"True = {len(df_include.loc[(df_include['Include'] == 'True')])}, False = {len(df_include.loc[(df_include['Include'] == 'False')])}")
df_include

True = 321, False = 84
True = 321, False = 208


,cat_name,app_id,name,search_details,counts,country_counts,urls,Include
0,Business,981163277,JOB TODAY: Easy Job Search,"find a job_20,glassdoor_20,indeed_20,job searc...",157,50,[https://apps.apple.com/ae/app/job-today-easy-...,True
1,Business,1040443949,Reed.co.uk Job Search,"find a job_20,glassdoor_20,indeed_20,job searc...",132,57,[https://apps.apple.com/ae/app/reed-co-uk-job-...,True
2,Business,589698942,Glassdoor - Job Search Tools,"find a job_20,glassdoor_20,indeed_20,job searc...",130,33,[https://apps.apple.com/ae/app/glassdoor-job-s...,True
3,Business,309735670,Indeed Job Search,"find a job_20,glassdoor_20,indeed_20,job searc...",128,32,[https://apps.apple.com/ae/app/indeed-job-sear...,True
4,Business,288429040,LinkedIn: Job Search & News,"find a job_20,glassdoor_20,indeed_20,job searc...",127,34,[https://apps.apple.com/ae/app/linkedin-job-se...,True
...,...,...,...,...,...,...,...,...
775,Photo & Video,1361012099,Facelab: Gezicht Foto Bewerken,job search_100,1,1,[https://apps.apple.com/nl/app/facelab-gezicht...,False
883,Travel,1492085245,Rida — cheaper than taxi ride,"find a job_20,job search_100",2,1,[https://apps.apple.com/ng/app/rida-cheaper-th...,False
884,Travel,475966832,同程旅行-订酒店机票火车票,job search_100,1,1,[https://apps.apple.com/hk/app/%E5%90%8C%E7%A8...,False
885,Travel,520472426,MyWheels - Auto huren,indeed_20,1,1,[https://apps.apple.com/nl/app/mywheels-auto-h...,False


### Preparing for DC 

In [27]:
df_appids_filt = df_include.copy().set_index('app_id')
case_specific_ui.df_to_sheet(df_appids_filt, sheet= 'SOURCE - Apps Overview', index=True, replace=True)
case_specific_ui.df_to_sheet(df_apps_normalized[['app_id', 'ranking', 'name', 'seller_name', 'url', 'Internal Country']], sheet= 'SOURCE - Apps Tot', index=False, replace=True)

### Import the DC and output a .csv with the included apps 

In [28]:
df_appids_dc = case_specific_ui.sheet_to_df(sheet="OUTPUT - Apps DC", index=True).reset_index()
df_appids_dc['app_id'] = df_appids_dc.app_id.astype(int)
df_appids_dc['Include'] = df_appids_dc['Include'].replace(r'^\s*$', np.nan, regex=True).str.capitalize()

df_apps_normalized_filt = df_apps_normalized.set_index('app_id').join(df_appids_dc[['app_id','Include', 'SubCat']].set_index('app_id')).sort_values(by='ranking').reset_index()#.drop_duplicates()
print(f"True = {len(df_apps_normalized_filt.loc[(df_apps_normalized_filt['Include'] == 'True')])}, False = {len(df_apps_normalized_filt.loc[(df_apps_normalized_filt['Include'] == 'False')])}, Empty = {len(df_apps_normalized_filt.loc[(df_apps_normalized_filt['Include'].isnull() == True)])}")
print("The included categories are:")
print(df_apps_normalized_filt.loc[df_apps_normalized_filt['Include'] != 'False']['cat_name'].value_counts())

results = path_results / f'df_{theme}_filtered.csv'
with open(results, 'w') as outfile:
    df_apps_normalized_filt.loc[df_apps_normalized_filt['Include'] != 'False'].to_csv(results, index=False)       

True = 6547, False = 1684, Empty = 464
The included categories are:
Business             6365
Lifestyle             181
Education             112
Productivity          100
Social Networking      74
Utilities              61
Reference              53
Finance                23
Medical                18
Shopping               11
Developer Tools        11
News                    1
Health & Fitness        1
Name: cat_name, dtype: int64


In [29]:
df_country_view_50 = (
    df_apps_normalized_filt.loc[(df_apps_normalized_filt['Include'] != 'False') & 
                                (df_apps_normalized_filt['ranking'] <= 50)]
    [['Internal Country', 'app_id']]
    .groupby('Internal Country').count()
    .sort_values(by='app_id', ascending=False)
    .rename(columns={'app_id':'ranking_below_50'})
)

df_country_view = (
    df_apps_normalized_filt.loc[(df_apps_normalized_filt['Include'] != 'False')]
    [['Internal Country', 'app_id']]
    .groupby('Internal Country').count()
    .sort_values(by='app_id', ascending=False)
    .rename(columns={'app_id':'apps_count'}))


df_country_view.join(df_country_view_50)

,apps_count,ranking_below_50
Internal Country,,
Austria,141,98
Singapore,139,94
UK,139,96
USA,138,96
Kenya,137,91
Norway,137,92
Australia,136,92
Bulgaria,134,93
Canada,134,92


## Step 3
### Scoring:
### Importing all the DC from the research_ui and the existing apps dataframes 

In [30]:
df_categories_ios = pd.read_csv(path_applyzer_markets_ios)
df_applyzer_ranking_all = pd.read_csv('df_applyzer_ranking_all.csv')
df_applyzer_apps_all = pd.read_csv('df_applyzer_apps_all.csv', index_col=0)
df_appscraper_all = df_results()

#### Creating a df with Applyzer apps and appending the new df from Appscraper

In [31]:
df_charts_ww = (
    pd.merge(df_applyzer_ranking_all, df_applyzer_apps_all, how='left', left_on='app_id', right_index=True)
    .set_index(['app_id'])
    .join(df_countries[['Internal Country', 'id', 'internet_users', 'internet_penetration']].set_index('id'), on='country_id')
    .join(df_categories_ios.rename(columns={'name':'cat_name'}).set_index('id'), on='category_id')
    .reset_index()
)
df_charts_ww

df_charts_ww_free = (
    pd
    .merge(df_applyzer_ranking_all[df_applyzer_ranking_all.chart == 'free'], 
             df_applyzer_apps_all, 
             how='left', left_on='app_id', right_index=True)
    .drop(columns=['chart'])
    .set_index(['app_id'])
    .join(df_countries[['Internal Country', 'id', 'internet_users', 'internet_penetration']].set_index('id'), on='country_id')
    .join(df_categories_ios.rename(columns={'name':'cat_name'}).set_index('id'), on='category_id')
    .reset_index()
)
df_charts_ww_free['ranking'] = df_charts_ww_free['ranking'] + 1
df_charts_ww_free.head()


,app_id,ranking,country_id,category_id,name,seller_name,url,Internal Country,internet_users,internet_penetration,cat_name
0,1527399597,1,2,4,PlantIn: Plant Identifier,Vortemol Limited,https://itunes.apple.com/app/id1527399597,Argentina,33712623.81,74.29%,Education
1,924620788,2,2,4,Google Classroom,Google LLC,https://itunes.apple.com/app/id924620788,Argentina,33712623.81,74.29%,Education
2,570060128,3,2,4,Duolingo - Language Lessons,"Duolingo, Inc",https://itunes.apple.com/app/id570060128,Argentina,33712623.81,74.29%,Education
3,1208138685,4,2,4,Toca Life World: Build stories,Toca Boca AB,https://itunes.apple.com/app/id1208138685,Argentina,33712623.81,74.29%,Education
4,919087726,5,2,4,Photomath,"Photomath, Inc.",https://itunes.apple.com/app/id919087726,Argentina,33712623.81,74.29%,Education


#### Introducing SubCategories and appending new dataframes

In [32]:
df_app_dc = research_ui.sheet_to_df(sheet="OUTPUT - Applyzer App DC", index=False)
df_app_dc['app_id'] = df_app_dc.app_id.astype(int)
df_app_dc.SubCat = df_app_dc.SubCat.replace(r'^\s*$', np.nan, regex=True)
df_app_dc = pd.merge(df_app_dc, df_countries, left_on='origin country', right_on='Internal Country', how='left')
df_app_dc = df_app_dc[['app_id', 'id', 'Internal Country', 'SubCat']].rename(columns={'id':'origin_id', 'Internal Country':'Origin Country'})

df_charts_ww_free = pd.merge(df_charts_ww_free, df_app_dc, left_on='app_id', right_on='app_id', how='left')


df_charts_ww_free = df_charts_ww_free.append(df_appscraper_all[['app_id', 'ranking', 'country_id', 'category_id', 'name',
       'seller_name', 'url', 'Internal Country', 'internet_users',
       'internet_penetration', 'cat_name', 'SubCat']]).reset_index(drop=True)

df_charts_ww_free.head()

,app_id,ranking,country_id,category_id,name,seller_name,url,Internal Country,internet_users,internet_penetration,cat_name,origin_id,Origin Country,SubCat
0,1527399597,1,2.0,4,PlantIn: Plant Identifier,Vortemol Limited,https://itunes.apple.com/app/id1527399597,Argentina,33712623.81,74.29%,Education,NaN,NaN,Reference
1,924620788,2,2.0,4,Google Classroom,Google LLC,https://itunes.apple.com/app/id924620788,Argentina,33712623.81,74.29%,Education,1.0,USA,Study Platform
2,570060128,3,2.0,4,Duolingo - Language Lessons,"Duolingo, Inc",https://itunes.apple.com/app/id570060128,Argentina,33712623.81,74.29%,Education,1.0,USA,Online Learning
3,1208138685,4,2.0,4,Toca Life World: Build stories,Toca Boca AB,https://itunes.apple.com/app/id1208138685,Argentina,33712623.81,74.29%,Education,54.0,Sweden,Edutainment
4,919087726,5,2.0,4,Photomath,"Photomath, Inc.",https://itunes.apple.com/app/id919087726,Argentina,33712623.81,74.29%,Education,1.0,USA,Learning Tools


#### Scoring

In [33]:
df_charts_ww_free_total = df_charts_ww_free[df_charts_ww_free.category_id == 99].copy()
df_charts_ww_free_total['score_ranking'] = 100 * (0.985 ** df_charts_ww_free_total['ranking'])
df_charts_ww_free_total['score_ranking_adj'] = df_charts_ww_free_total['score_ranking'].astype(float) * df_charts_ww_free_total['internet_users'].astype(float)
df_charts_ww_free_cat = df_charts_ww_free[df_charts_ww_free.category_id != 99].copy()
df_charts_ww_free_total['ranking_overall'] = df_charts_ww_free_total['ranking']
df_charts_ww_free_cat.app_id = df_charts_ww_free_cat.app_id.astype("int64")

df_charts_ww_free_cat = pd.merge(
    df_charts_ww_free_cat,
    df_charts_ww_free_total[['country_id', 'app_id','score_ranking','score_ranking_adj', 'ranking_overall']],
    left_on=['country_id', 'app_id'],
    right_on=['country_id', 'app_id'],
    how='left'
)

df_country_count = df_charts_ww_free_cat[df_charts_ww_free_cat.ranking <= 50].groupby(['app_id'])[['country_id']].agg('count').sort_values('country_id', ascending=False)
df_country_count['unique_score'] = 1 / (1.04 ** df_country_count.country_id)
df_country_count = df_country_count.reset_index()


df_charts_ww_final = pd.merge(df_charts_ww_free_cat, df_country_count[['app_id', 'unique_score']], left_on='app_id', right_on='app_id', how='left')
df_charts_ww_final.loc[df_charts_ww_final.origin_id == df_charts_ww_final.country_id, 'unique_score'] = 1 / (1.04)
df_charts_ww_final

,app_id,ranking,country_id,category_id,name,seller_name,url,Internal Country,internet_users,internet_penetration,cat_name,origin_id,Origin Country,SubCat,score_ranking,score_ranking_adj,ranking_overall,unique_score
0,1527399597,1,2.0,4,PlantIn: Plant Identifier,Vortemol Limited,https://itunes.apple.com/app/id1527399597,Argentina,33712623.81,74.29%,Education,NaN,NaN,Reference,51.427349,1.733751e+09,44.0,0.390121
1,924620788,2,2.0,4,Google Classroom,Google LLC,https://itunes.apple.com/app/id924620788,Argentina,33712623.81,74.29%,Education,1.0,USA,Study Platform,48.410444,1.632043e+09,48.0,0.164614
2,570060128,3,2.0,4,Duolingo - Language Lessons,"Duolingo, Inc",https://itunes.apple.com/app/id570060128,Argentina,33712623.81,74.29%,Education,1.0,USA,Online Learning,26.850480,9.052001e+08,87.0,0.111207
3,1208138685,4,2.0,4,Toca Life World: Build stories,Toca Boca AB,https://itunes.apple.com/app/id1208138685,Argentina,33712623.81,74.29%,Education,54.0,Sweden,Edutainment,22.396844,7.550564e+08,99.0,0.140713
4,919087726,5,2.0,4,Photomath,"Photomath, Inc.",https://itunes.apple.com/app/id919087726,Argentina,33712623.81,74.29%,Education,1.0,USA,Learning Tools,20.148394,6.792552e+08,106.0,0.120282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695629,1512048310,20,10.0,6015,Crypto.com l DeFi Wallet,DeFi Labs,https://apps.apple.com/co/app/crypto-com-l-def...,Colombia,33077385.89,65.01%,Finance,NaN,NaN,Cryptocurrency,NaN,NaN,NaN,0.513373
695630,1205990992,20,62.0,6015,TradingView - FX、株価チャート・ビットコイン,"TradingView, Inc.",https://apps.apple.com/jp/app/tradingview-fx-%...,Japan,114865144.0,91.28%,Finance,NaN,NaN,Cryptocurrency,NaN,NaN,NaN,0.050754
695631,1509163188,20,48.0,6015,VC Wallet,BLOCK CHAIN JAPAN K.K.,https://apps.apple.com/ch/app/vc-wallet/id1509...,Switzerland,8044930.658,93.15%,Finance,NaN,NaN,Cryptocurrency,NaN,NaN,NaN,0.888996
695632,1506274532,20,34.0,6015,BlockFi: Buy and Trade Crypto,"BlockFi, Inc.",https://apps.apple.com/mx/app/blockfi-buy-and-...,Mexico,90343064.59,70.07%,Finance,NaN,NaN,Cryptocurrency,NaN,NaN,NaN,0.924556


## Step 4
### Grouping and Scoring the apps, by Subcategories

#### SubCategories Overview

In [34]:
df_sub_cat = (
    df_charts_ww_final[['SubCat','cat_name','app_id']]
    .groupby('SubCat').agg({'app_id':'count', 'cat_name':lambda x: ', '.join(sorted(pd.Series.unique(x)))})
    .sort_values(by='app_id', ascending=False)
    .rename(columns={'app_id':'counts',
                    'cat_name': 'cat_list'})
)
df_sub_cat['cat_list'] = df_sub_cat['cat_list'].astype(str)
df_sub_cat['cat_list'] = df_sub_cat['cat_list'].str.strip('{}')

creative_ui.df_to_sheet(df_sub_cat, sheet='SOURCE - Available SubCat', index=True)
df_sub_cat

,counts,cat_list
SubCat,,
Health Self Management,7776,"Education, Healthcare/Fitness, Medical, Naviga..."
Job Search,7011,"Business, Developer Tools, Education, Finance,..."
Productivity,6092,"Education, Healthcare/Fitness, Medical, Naviga..."
Travel Booking,5156,"Navigation, Travel"
Maps,4918,"Navigation, Travel"
Cryptocurrency,4034,"Business, Education, Entertainment, Finance, L..."
Government/COVID,3031,"Healthcare/Fitness, Medical, Navigation, Trave..."
Delivery Service,2796,"Food & Drink, Navigation, Travel, Utilities"
Personal Transport,2723,"Healthcare/Fitness, Navigation, Travel"


#### Ranking score by Category and SubCategory

In [35]:
df_ranking_score = df_charts_ww_final.copy()
df_ranking_score = (
    df_ranking_score
    [['Internal Country','cat_name', 'SubCat','internet_penetration', 'unique_score', 'score_ranking','app_id']]
)

df_ranking_score['internet_penetration'] = df_ranking_score['internet_penetration'].str.replace('%','')
df_ranking_score['internet_penetration'] = df_ranking_score['internet_penetration'].replace(r'^\s*$', 0, regex=True).astype(float)
df_ranking_score = df_ranking_score.fillna(0)
df_ranking_score['final_score'] = df_ranking_score['score_ranking'] * df_ranking_score['internet_penetration']/ 100
df_ranking_score = df_ranking_score[df_ranking_score.final_score != 0]
df_ranking_score.SubCat = df_ranking_score.SubCat.replace(0, np.nan)

df_cat_scores = df_ranking_score.groupby(['Internal Country', 'cat_name']).agg('sum')[['final_score']].sort_values('final_score', ascending=False).unstack()#.droplevel(0, axis=1)
#df_cat_scores.sort_values(by=('Business'), ascending=False)

df_subcat_scores = df_ranking_score.groupby(['Internal Country', 'SubCat']).agg('sum')[['final_score']].sort_values('final_score', ascending=False).unstack()#.droplevel(0, axis=1)
#df_subcat_scores

In [36]:
cat_score = df_cat_scores.transpose().droplevel(0)
cat_score['score'] = cat_score.sum(axis=1)
cat_score.sort_values(by='score', ascending=False)[['score']]

Internal Country,score
cat_name,
Finance,23175.355271
Utilities,22761.358424
Travel,19759.084892
Food & Drink,13729.999521
Healthcare/Fitness,10429.692656
Education,10184.410434
Navigation,10071.814819
Medical,3743.369116
Business,3528.573699


In [37]:
subcat_score = df_subcat_scores.transpose().droplevel(0)
subcat_score['score'] = subcat_score.sum(axis=1)

creative_ui.df_to_sheet(subcat_score.sort_values(by='score', ascending=False)[['score']], sheet='SOURCE - Available SubCat Scored', index=True)
subcat_score.sort_values(by='score', ascending=False)[['score']]

Internal Country,score
SubCat,
Productivity,17468.953174
Personal Transport,8452.340051
Maps,8391.000552
Travel Booking,7264.548127
Delivery Service,7184.896037
Health Self Management,6105.442941
Cryptocurrency,4970.095600
Government/COVID,4876.957668
Job Search,3790.312150


### Importing the selected sub categories from creative.ui and scoring the customised category

In [47]:
df_custom = creative_ui.sheet_to_df(sheet="OUTPUT - Selected SubCat", index=False)
custom_list = df_custom['Selected_SubCat'].tolist()
digital_learning = custom_list
custom_cat = 'digital_learning'
digital_learning

['Study Platform',
 'Online Learning',
 'Learning Tools',
 'Student Platform',
 '-',
 'Classroom']

In [48]:
df_ranking_score = df_charts_ww_final.copy()
df_ranking_score = (
    df_ranking_score
    [['Internal Country','SubCat', 'internet_penetration' , 'unique_score', 'score_ranking','app_id']]
)
df_ranking_score['final_score'] = df_ranking_score['score_ranking'].fillna(0) * df_ranking_score['internet_penetration'].str.replace('%','').astype(float) / 100
df_ranking_score = df_ranking_score[df_ranking_score.final_score.notnull()]
df_ranking_score['custom_cat'] = df_ranking_score.SubCat.isin(custom_list)
df_custom_scores = df_ranking_score.groupby(['Internal Country', 'custom_cat']).agg('sum')[['final_score']].sort_values('final_score', ascending=False).unstack().droplevel(0, axis=1)
creative_ui.df_to_sheet(df_custom_scores.sort_values(by=True, ascending=False), sheet=f'SOURCE - {custom_cat} Scored', index=True, replace=True)
creative_ui.df_to_sheet(df_custom_scores.sort_values(by=True, ascending=False), sheet= 'SOURCE - Custom Scored', index=True, replace=True)

df_custom_scores.sort_values(by=True, ascending=False)

custom_cat,False,True
Internal Country,,
Sweden,2637.108597,424.436497
Norway,3265.341093,423.799236
Switzerland,2496.729473,272.310614
USA,1825.340290,247.304726
New Zealand,1757.617943,233.351231
Finland,2146.437758,232.029099
Denmark,2658.593907,231.407452
Netherlands,2733.917073,183.152389
Estonia,2478.570806,159.617967
